In [5]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [40]:
# List all experiments
experiments = client.search_experiments()
for experiment in experiments:
    print(f"Experiment ID: {experiment.experiment_id}, Name: {experiment.name}")

Experiment ID: 2, Name: my-cool-experiment
Experiment ID: 1, Name: first-experiment
Experiment ID: 0, Name: Default


In [3]:
client.create_experiment(name="my-cool-experiment")

'2'

In [12]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.33",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [13]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: bebfe2de11a040be99e26c09639aec27, rmse: 6.3250
run id: 82bf887e4d1c45fd8ff7b50ec39fb6f6, rmse: 6.3289


# Interacting with the Model Registry

In [14]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [28]:
run_id = "e175c3ef90ea4ff1ab3a186bc4738ace"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1718261281669, current_stage='None', description=None, last_updated_timestamp=1718261281669, name='nyc-taxi-regressor', run_id='e175c3ef90ea4ff1ab3a186bc4738ace', run_link=None, source='/workspaces/mlops-practice/02-exp-tracking/mlruns/1/e175c3ef90ea4ff1ab3a186bc4738ace/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [16]:
model_uri

'runs:/bebfe2de11a040be99e26c09639aec27/model'

In [29]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: Archived
version: 4, stage: None


/tmp/ipykernel_34133/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.2/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [30]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_34133/4090915774.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1718259332934, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2024-06-13', last_updated_timestamp=1718261297272, name='nyc-taxi-regressor', run_id='e175c3ef90ea4ff1ab3a186bc4738ace', run_link='', source='/workspaces/mlops-practice/02-exp-tracking/mlruns/1/e175c3ef90ea4ff1ab3a186bc4738ace/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'sklearn-linearregressin'}, user_id=None, version=2>

In [31]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1718259332934, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2024-06-13', last_updated_timestamp=1718261299303, name='nyc-taxi-regressor', run_id='e175c3ef90ea4ff1ab3a186bc4738ace', run_link='', source='/workspaces/mlops-practice/02-exp-tracking/mlruns/1/e175c3ef90ea4ff1ab3a186bc4738ace/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'sklearn-linearregressin'}, user_id=None, version=2>

In [21]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df_filter = df.copy()
    df_filter = df_filter.sample(n=50000).reset_index(drop=True)
    
    df_filter['duration'] = df_filter.tpep_dropoff_datetime -  df_filter.tpep_pickup_datetime
    df_filter['duration'] = df_filter.duration.apply(lambda td: td.total_seconds()/ 60)

    df_filter = df_filter[(df_filter.duration > 0) & (df_filter.duration <= 120)]
    df_filter = df_filter[df_filter.passenger_count > 0]
    df_filter = df_filter[df_filter.fare_amount > 0]
    df_filter = df_filter[df_filter.trip_distance > 0]
    
    df_filter.reset_index(drop=True)
    
    categorical = ['PULocationID','DOLocationID']
    numerical = ['trip_distance']
    
    df_filter[categorical] = df_filter[categorical].astype(str)
    
    return df_filter


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [22]:
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet")

In [32]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-practice/02-exp-tracking/preprocessor'

In [33]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [34]:

X_test = preprocess(df, dv)

In [35]:
target = "duration"
y_test = df[target].values

In [37]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [06:50:38] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 4.68 s, sys: 75 ms, total: 4.76 s
Wall time: 3.07 s


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.110828094650305}

In [38]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 60.4 ms, sys: 0 ns, total: 60.4 ms
Wall time: 64.7 ms


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.442475193233695}

In [39]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_34133/1703833061.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1718259332934, current_stage='Production', description='The model version 2 was transitioned to Staging on 2024-06-13', last_updated_timestamp=1718261521186, name='nyc-taxi-regressor', run_id='e175c3ef90ea4ff1ab3a186bc4738ace', run_link='', source='/workspaces/mlops-practice/02-exp-tracking/mlruns/1/e175c3ef90ea4ff1ab3a186bc4738ace/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'sklearn-linearregressin'}, user_id=None, version=2>